In [1]:
import os
import cv2
import imageio
import numpy as np
from typing import List
import glob

In [2]:
def load_images(filenames: List) -> List:
    return [cv2.imread(filename) for filename in filenames]

In [3]:
imgs_paths = glob.glob("./Patterns/*.jpg")

In [4]:
imgs_paths

['./Patterns\\circle.jpg',
 './Patterns\\circle_line.jpg',
 './Patterns\\circle_scaled.jpg',
 './Patterns\\line.jpg',
 './Patterns\\pattern_0.jpg',
 './Patterns\\pattern_1.jpg',
 './Patterns\\pattern_2.jpg',
 './Patterns\\pattern_3.jpg',
 './Patterns\\pattern_4.jpg',
 './Patterns\\pattern_5.jpg',
 './Patterns\\pattern_7.jpg',
 './Patterns\\pattern_9.jpg',
 './Patterns\\square.jpg',
 './Patterns\\square_line.jpg']

In [5]:
imgs = load_images(imgs_paths)

In [6]:
imgs

[array([[[59, 58, 60],
         [58, 57, 59],
         [57, 59, 60],
         ...,
         [57, 62, 63],
         [58, 63, 64],
         [59, 64, 65]],
 
        [[58, 57, 59],
         [55, 57, 58],
         [55, 57, 58],
         ...,
         [58, 63, 64],
         [57, 62, 63],
         [56, 61, 62]],
 
        [[57, 59, 60],
         [56, 58, 59],
         [54, 56, 57],
         ...,
         [58, 63, 64],
         [56, 61, 62],
         [53, 58, 59]],
 
        ...,
 
        [[35, 50, 53],
         [34, 49, 52],
         [36, 48, 52],
         ...,
         [71, 83, 85],
         [70, 82, 84],
         [68, 81, 83]],
 
        [[38, 50, 54],
         [36, 48, 52],
         [35, 46, 50],
         ...,
         [69, 82, 84],
         [67, 82, 84],
         [66, 81, 83]],
 
        [[39, 51, 55],
         [36, 48, 52],
         [34, 45, 49],
         ...,
         [67, 82, 84],
         [68, 83, 85],
         [65, 83, 84]]], dtype=uint8),
 array([[[36, 53, 50],
         [39, 56, 5

In [7]:
def show_image(img: np.array, img_name: str = "Image"):
    cv2.imshow(img_name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def write_image(output_folder: str, img_name: str, img: np.array):
    os.makedirs(output_folder, exist_ok=True)
    img_path = os.path.join(output_folder, img_name)
    cv2.imwrite(img_path, img)

In [8]:
# for i, img in enumerate(imgs):
#     show_image(img,imgs_paths[i])

In [9]:
hsv_imgs = [cv2.cvtColor(img, cv2.COLOR_BGR2HSV) for img in imgs]

# for img in hsv_imgs:
#     show_image(img)

In [10]:
from utils import non_max_suppression, get_hsv_color_ranges

In [11]:
# img_test = load_images(["./Patterns/circle_scaled.jpg"])
# img_test = img_test[0]
# show_image(img_test)

In [12]:
# get_hsv_color_ranges(img_test)

In [11]:
lower_black = (0, 90, 0)
upper_black = (255, 255, 255)

black_masks = [cv2.inRange(hsv_img, lower_black, upper_black) for hsv_img in hsv_imgs]

In [13]:
black_masks

[array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
      

In [16]:
for mask in black_masks:
    show_image(np.asarray(mask,dtype=np.float32),"Mask")

In [14]:
imgs_segmented = [cv2.bitwise_and(img, img, mask=mask) for img, mask in zip(imgs, black_masks)]

In [15]:
for i, img in enumerate(imgs_segmented):
    show_image(img, imgs_paths[i])

In [20]:
def custom_dilate(img):
    # TODO pad the original image so it can keep dimensions after processing
    padded = np.pad(img, pad_width=1,mode='constant', constant_values=0)
    
    # TODO get img shape
    width = padded.shape[1]
    height = padded.shape[0]
    
    # TODO Create an element with the same dimensions as the padded img
    dilated = np.zeros((height, width), dtype=np.uint8)

    for j in range(1,height-1):
        for i in range(1,width-1):
            # TODO Add logic to the operation
            structuring_element = padded[j-1:j+2, i-1:i+2]
            
            dilated[j,i] = np.max(structuring_element)
            
    # TODO Select the region of interest (ROI). Modify if needed
    dilated = dilated[1:height+1, 1:width+1]
    
    return dilated

In [21]:
def custom_erode(img):
    # TODO pad the original image so it can keep dimensions after processing
    padded = np.pad(img, pad_width=1,mode='constant', constant_values=0)
    
    # TODO get img shape
    width = padded.shape[1]
    height = padded.shape[0]
    
    # TODO Create an element with the same dimensions as the padded img
    eroded = np.zeros((height, width), dtype=np.uint8)
    
    for j in range(1,height-1):
        for i in range(1,width-1):
            # TODO Add logic to the operation
            structuring_element = padded[j-1:j+2, i-1:i+2]

            eroded[j,i] = np.min(structuring_element)
            
    # TODO Select the region of interest (ROI). Modify if needed
    eroded = eroded[1:height+1, 1:width+1]
    
    return eroded

In [22]:
# dilated_masks = [custom_dilate(black_mask) for black_mask in black_masks]

In [26]:
# for mask in dilated_masks:
#     show_image(np.asarray(mask, dtype=np.float32),"Mask")

In [31]:
show_image(np.asarray(black_masks[0]),"Mask")

In [77]:
kernel = np.ones((2,2),np.uint8)
gradient = cv2.morphologyEx(black_masks[0], cv2.MORPH_GRADIENT, kernel)

In [78]:
show_image(np.asarray(gradient),"Gradient")

In [79]:
border_color = [0,255,0]
colored_border = np.zeros_like(img,np.uint8)
colored_border[:] = border_color

border = cv2.bitwise_and(colored_border,colored_border,mask=gradient)

In [80]:
show_image(np.asarray(border,np.float32),"Mask")

In [81]:
result = cv2.add(imgs[0], border)

In [82]:
show_image(np.asarray(result),"Result")

In [23]:
kernel = np.ones((300,300),np.float32)
morph_open_masks = [cv2.morphologyEx(np.asarray(black_mask,dtype=np.float32), cv2.MORPH_OPEN, kernel) for black_mask in black_masks] # Aplico el operador de apertura con el kernel

In [24]:
morph_open_masks

[array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],


In [25]:
for mask in morph_open_masks:
    show_image(np.asarray(mask,dtype=np.float32))

In [19]:
border_masks = [cv2.subtract(morph_open_mask, black_mask) for morph_open_mask, black_mask in zip(morph_open_masks,black_masks)]

In [20]:
border_masks

[array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0

In [21]:
border_color = [0,255,0] # BGR
colored_borders = [np.zeros_like(img,np.uint8) for img in imgs]
for i, border in enumerate(colored_borders):
    border[:] = border_color

colored_border_masks = [cv2.bitwise_and(colored_border,colored_border,mask=mask) for colored_border, mask in zip(colored_borders,border_masks)]

In [22]:
final_results = [cv2.add(img, border_mask) for img, border_mask in zip(imgs, colored_border_masks)]

In [23]:
output_folder = "./patterns_with_borders"


for i, result in enumerate(final_results):
    show_image(result, imgs_paths[i])
    write_image(output_folder, f"Final_{os.path.basename(imgs_paths[i])}", result)

In [24]:
imgs[0].shape

(720, 1280, 3)

In [98]:
# Convertir a escala de grises
gray_img = cv2.cvtColor(imgs[0], cv2.COLOR_BGR2GRAY)

#smoothed_img = cv2.GaussianBlur(black_masks[0], (3, 3), 0)  # (5, 5) es el tamaño del kernel
kernel = np.ones((3, 3), np.uint8)
eroded = cv2.erode(black_masks[0], kernel, iterations=1)
dilated = cv2.dilate(eroded,kernel,iterations=1)

# Aplicar Sobel en x e y
sobel_x = cv2.Sobel(dilated, cv2.CV_64F, 1, 0, ksize=3)
sobel_y = cv2.Sobel(dilated, cv2.CV_64F, 0, 1, ksize=3)

# Calcular magnitud del gradiente
sobel_combined = cv2.magnitude(sobel_x, sobel_y)
sobel_combined = np.uint8(sobel_combined)

# Umbralizar para bordes claros
_, binary_edges = cv2.threshold(sobel_combined, 50, 255, cv2.THRESH_BINARY)

In [102]:
border_color = [0,255,0]
colored_border = np.zeros_like(img,np.uint8)
colored_border[:] = border_color

border = cv2.bitwise_and(colored_border,colored_border,mask=binary_edges)

result = cv2.add(imgs[0],border)

In [103]:
show_image(result,"Image")

In [123]:
from skimage import filters, feature

In [135]:
img_mask = feature.canny(black_masks[0])
img_mask = img_mask.astype(np.uint8) * 255 #para poder representarlo

img_with_borders = imgs[0].copy()

# Encontrar las coordenadas de los bordes
# Los bordes de Canny son 255, y el fondo es 0 en la máscara binaria
borders = np.where(img_mask == 255)

# Dibujar los bordes en verde sobre la imagen original
for y, x in zip(borders[0], borders[1]):
    img_with_borders[y, x] = [0, 255, 0]  # Establecer los píxeles en verde (BGR)

# Mostrar la imagen con los bordes en verde
cv2.imshow("Imagen con Bordes Verdes", img_with_borders)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [134]:
show_image(img_mask)

In [126]:
border_color = [0,255,0]
colored_border = np.zeros_like(imgs[0],np.uint8)
colored_border[:] = border_color

In [127]:
border = cv2.bitwise_and(colored_border,colored_border,mask=img_mask)
border

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:230: error: (-215:Assertion failed) (mtype == CV_8U || mtype == CV_8S) && _mask.sameSize(*psrc1) in function 'cv::binary_op'


In [122]:




border = cv2.bitwise_and(colored_border,colored_border,mask=img_mask)

result = cv2.add(imgs[0],border)

#img_prewitt = filters.prewitt(black_masks[0])
#img_canny = feature.canny(black_masks[0])
#img_canny = img_canny.astype(np.uint8) * 255 #para poder representarlo
show_image(result, img_name="Bordes Sobel con scikit-image")
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:230: error: (-215:Assertion failed) (mtype == CV_8U || mtype == CV_8S) && _mask.sameSize(*psrc1) in function 'cv::binary_op'


Probar con skimage 

### MatchTemplate()

In [120]:
# Cargar la imagen principal y la plantilla (template)
img = cv2.imread('./Patterns/circle.jpg')  # Imagen en la que buscarás el patrón
template = cv2.imread('./Patterns/circle_pattern.jpg')  # Plantilla que buscas en la imagen

In [121]:
# Convertir las imágenes a escala de grises (opcional, pero recomendado para facilitar la comparación)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

# Aplicar matchTemplate para buscar la plantilla en la imagen
result = cv2.matchTemplate(img_gray, template_gray, cv2.TM_CCOEFF_NORMED)

# Encontrar la ubicación de la mejor coincidencia
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

# Si el valor máximo es mayor que un umbral, consideramos que hay coincidencia
threshold = 0.8  # Puedes ajustar este umbral
if max_val >= threshold:
    print("Se ha encontrado una coincidencia.")
    # Dibujar un rectángulo alrededor de la coincidencia
    top_left = max_loc
    h, w = template.shape[:2]
    bottom_right = (top_left[0] + w, top_left[1] + h)
    cv2.rectangle(img, top_left, bottom_right, (0, 255, 0), 2)

# Mostrar la imagen con la coincidencia
cv2.imshow('Resultado', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Se ha encontrado una coincidencia.
